In [70]:
import pygame
import torch
from torch import nn
import numpy as np
import time

In [71]:
#make x-way xor data
x=8
x_data=[[int(dig) for dig in (str(bin(i))[2:].rjust(x,"0"))] for i in range((2**x))]
y_data=torch.tensor([i&1 for i in range(2**x)])
y_data=y_data.float().unsqueeze(1)
numpy_array = np.array(x_data, dtype=np.float32)
x_data = torch.from_numpy(numpy_array)


In [72]:
class layer_visual():
    @staticmethod
    def get_visual(screen,layer,x,font):
        d=600//len(layer)
        for i,l in enumerate(layer):
            try:
                pygame.draw.circle(screen,(20,20,200),(x,(100+(i*d))),50,25)
                ts=font.render(str(l),True,(255,255,255))
                screen.blit(ts,(x-5,(90+(i*d))))        
            except:
                pass
    @staticmethod
    def map_weight_to_color(weight):
        if weight>0:
            return (0,0,min(50+(50*weight),255))
        if weight==0:
            return (50,50,50)
        return (min(50+(50*-weight),255),0,0)
    @staticmethod
    def render_weight_lines(weights,x1,x2,screen):
        for l1 in range(len(weights)):
            for l2 in range(len(weights[0])):
                pygame.draw.line(screen,
                layer_visual.map_weight_to_color(weights[l1][l2]),
                (x2,(100+(l1*(600//len(weights))))),
                (x1,(100+(l2*(600//len(weights[0]))))), 7)
    @staticmethod
    def display_accuracy(accuracy,screen,font):
        ts=font.render(f"Accuracy: {accuracy}%",True,(0,0,255))
        screen.blit(ts,(700,600))

class Model(nn.Module):
    def __init__(self,_layer_sizes):
        super(Model, self).__init__()
        self.length=len(_layer_sizes)
        self.layers=[]
        self.layer_sizes=_layer_sizes
        i=0
        while i<len(_layer_sizes)-1:
            self.layers.append(nn.Linear(_layer_sizes[i],_layer_sizes[i+1]))
            if i!=len(_layer_sizes)-1:
                self.layers.append(nn.Tanh())
            i+=1
        self.layers.pop(-1)
        self.layers.append(nn.Sigmoid())
        self.layers=nn.ModuleList(self.layers)
        self.device= "cuda" if torch.cuda.is_available() else "cpu"
    def __len__(self):
        return self.length
    def len_layer(self,layer):
        return self.layer_sizes[layer]
    def forward(self,x):
        for i in self.layers:
            x=i(x)
        return x
    def get_weights(self,layer):
        try:
            layer=self.layers[2*layer]
        except IndexError:
            return torch.tensor([])
        return layer.weight
class reset_button():
    @staticmethod
    def check(x_pos,y_pos):
        if 100>x_pos or 200<x_pos:
            return False
        return 700<=y_pos and 750>=y_pos
        
    @staticmethod
    def render(screen):
        pygame.draw.rect(screen,(200,30,30),(100,700,100,50))
        ts=font.render(f"Reset",True,(0,0,255))
        screen.blit(ts,(120,705))
class second_layer_amount():
    def __init__(self):
        self.amount=4
    def render(self,screen):
        pygame.draw.rect(screen,(30,30,200),(250,700,50,50))
        ts=font.render("-",True,(120,120,120))
        screen.blit(ts,(270,705))

        ts=font.render(str(self.amount),True,(30,30,200))
        screen.blit(ts,(320,705))

        pygame.draw.rect(screen,(30,30,200),(350,700,50,50))
        ts=font.render("+",True,(120,120,120))
        screen.blit(ts,(370,705))
    def process_click(self,x_pos,y_pos):
        if (not (250>x_pos or 300<x_pos)) and 700<=y_pos and 750>=y_pos and self.amount!=1:
            self.amount-=1
        if (not (350>x_pos or 400<x_pos)) and 700<=y_pos and 750>=y_pos:
            self.amount+=1
class third_layer_amount():
    def __init__(self):
        self.amount=4
    def render(self,screen):
        pygame.draw.rect(screen,(30,30,200),(550,700,50,50))
        ts=font.render("-",True,(120,120,120))
        screen.blit(ts,(570,705))

        ts=font.render(str(self.amount),True,(30,30,200))
        screen.blit(ts,(620,705))

        pygame.draw.rect(screen,(30,30,200),(650,700,50,50))
        ts=font.render("+",True,(120,120,120))
        screen.blit(ts,(670,705))
    def process_click(self,x_pos,y_pos):
        if (not (550>x_pos or 600<x_pos)) and 700<=y_pos and 750>=y_pos and self.amount!=1:
            self.amount-=1
        if (not (650>x_pos or 700<x_pos)) and 700<=y_pos and 750>=y_pos:
            self.amount+=1



In [73]:
pygame.init()
pygame.font.init()
font=pygame.font.SysFont('Arial',30)
#font2=pygame.font.SysFont('Arial',50)
WIDTH, HEIGHT = 1000, 800
screen = pygame.display.set_mode((WIDTH, HEIGHT))
pygame.display.set_caption("Visual AI")
model=Model([x,4,4,1])
print(model.get_weights(0))
running=True
FPS=40#40 epochs / frames per second
loss_fn=nn.BCELoss()
optim=torch.optim.SGD(params=model.parameters(),lr=0.02)
sla=second_layer_amount()
tla=third_layer_amount()
while running:
    for event in pygame.event.get():
        if event.type==pygame.QUIT:
            pygame.quit()
            running=False
        if event.type==pygame.MOUSEBUTTONDOWN:
            pos_x,pos_y=event.pos
            sla.process_click(pos_x,pos_y)
            tla.process_click(pos_x,pos_y)
            if reset_button.check(pos_x,pos_y):
                print("hi")
                model=Model([x,sla.amount,tla.amount,1])
                loss_fn=nn.BCELoss()
                optim=torch.optim.SGD(params=model.parameters(),lr=0.02)
    for i in range(len(model)):
        ls=model.len_layer(i)
        layer_visual.get_visual(screen,["" for i in range(ls)],200+i*175,font)
        layer_visual.render_weight_lines(model.get_weights(i),200+i*175,375+i*175,screen)
    try:pygame.display.flip()
    except:pass
    screen.fill((128,128,128))
    #train
    model.train()
    out=model(x_data)
    loss=loss_fn(out,y_data)
    loss.backward()
    optim.step()
    optim.zero_grad()
    model.eval()
    correct = (torch.abs(out - y_data)<0.5).sum().item()
    layer_visual.display_accuracy(str(int(correct)/int(y_data.shape[0])*100),screen,font)
    reset_button.render(screen)
    sla.render(screen)
    tla.render(screen)
    time.sleep(1/FPS)

Parameter containing:
tensor([[-0.0275,  0.0992, -0.1246, -0.1545, -0.0032, -0.2282,  0.2936, -0.0755],
        [-0.1056, -0.2880,  0.0472, -0.3306, -0.1679,  0.0705, -0.1631,  0.1987],
        [-0.0778, -0.2033,  0.2362, -0.0505, -0.2390, -0.3124, -0.1187, -0.1353],
        [ 0.1903, -0.2285,  0.1030, -0.1458,  0.3254,  0.2130, -0.1347,  0.2850]],
       requires_grad=True)
hi


error: display Surface quit